# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [4]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [5]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)

In [6]:
talks

,title,type,url_slug,venue,date,location,abstract_url,description,poster_url
0,Tarski Sheaves,Talk,albany,Applied Topology in Albany Seminar,2021-02-12,"Albany, New York",https://atia-seminar.github.io/flyers/210212-s...,"I gave a talk on our paper ""Cellular Sheaves o...",NaN
1,A Lattice-Theoretic Laplacian for Cellular She...,Talk,siam-cse21,SIAM Conference on Computational Science and E...,2021-03-01,"Fort Worth, Texas",NaN,I presented work on the Tarski Laplacian for t...,NaN
2,Cellular Sheaves and the Tarski Laplacian,Talk,abramsky,"Quantum Group, Oxford Department of Computer S...",2020-07-31,"Oxford, UK",NaN,"At the kind invitation of Vidit Nanda, I gave ...",NaN
3,A Novel Graph Laplacian for Lattice-Valued She...,Talk,siam-mds20,SIAM Conference on Mathematics of Data Science,2020-05-06,"Cincinnati, Ohio",https://meetings.siam.org/sess/dsp_talk.cfm?p=...,I presented work on the Tarski Laplacian for t...,NaN
4,Multidimensional Persistence Module Classifica...,Spotlight,neurips20,NeurIPS 2020 Worshop: Topological Data Analysi...,2020-12-11,"Vancouver, Canada",NaN,Multiparameter persistent homology has been la...,https://openreview.net/attachment?id=CqFcRp-_m...
5,Network Sheaves Valued in Categories of Adjunc...,Talk,act21,Applied Category Theory 2021,2021-07-12,"Cambridge, UK",https://hansriess.com/files/adjunctions.pdf,We report past and ongoing efforts to compute ...,NaN


In [7]:
talks.loc[5] = ['Network Sheaves Valued in Categories of Adjunctions and their Laplacians',
'Talk',
'act21',
'Applied Category Theory 2021',
'2021-07-12',
'Cambridge, UK',
'https://hansriess.com/files/adjunctions.pdf',
'We report past and ongoing efforts to compute (global) sections of cellular sheaves valued in categories of adjunctions. First, we discuss previous work on sheaves valued in the category of complete lattices and Galois connections. Then, we survey ongoing efforts to generalize the fixed point theorem (Theorem 1) to (i) the category of (sufficiently small) categories and adjunctions, and (ii) the category of ordered monoid -enriched categories and adjunctions.',
'']

In [7]:
talks.assign(video_embed=[
'<iframe width="560" height="315" src="https://www.youtube.com/embed/rRz1RcEiRPQ" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>',
'',
'',
'',
'<iframe src="//player.crossminds.ai/index.html?vid=606fe4b3f43a7f2f827c0451&utm_medium=share&utm_source=embed" scrolling="no" border="0" frameborder="no" framespacing="0" allowfullscreen="true"></iframe>',
''])

ValueError: Length of values does not match length of index

In [9]:
talks.to_csv("talks.tsv", sep="\t", index=False)

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [10]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.abstract_url)) > 3:
        md += "\n[Abstract](" + item.abstract_url + ")\n"
        
    if len(str(item.poster_url)) > 3:
        md += "\n[Poster](" + item.poster_url + ")\n"

    if len(str(item.video_url)) > 3:
        md += "\n[Poster](" + item.poster_url + ")\n"
        
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.